In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import sys  
import tifffile as tf
from tqdm import tqdm
import os
import cv2
from scipy.signal import butter, filtfilt, detrend

In [ ]:
module_path = r"C:\Users\alexc\Documents\GitHub\Wide-Brain"
current_path = os.getcwd()
sys.path.insert(1, module_path)

In [15]:
from WFmovie_mod import WFmovie
from WFmovie_mod import create_channel
from WFmovie_mod import ioi_epsilon_pathlength

In [16]:
def identify_folders(path, keywords):
    initial_folders = [f.path for f in os.scandir(path) if f.is_dir()]
    folders = []
    for folder in initial_folders:
        if all(keyword in folder for keyword in keywords):
            folders.append(folder)
    for i in range(len(folders)):
        folders[i] += '/'
    return folders

In [17]:
def identify_files(path, keywords):
    items = os.listdir(path)
    files = []
    for item in items:
        if all(keyword in item for keyword in keywords):
            files.append(item)
    files.sort()
    return files

In [18]:
def regress_timeseries(y, x, intercept=False, offset=False):
    """ Regress out the linear contribution of a regressor on time series data
    Args:
        y (ndarray): Time series specified as Nx1 vector. For M time series, specify as NxM array. 
        x (1D vector): NX1 time series to regress out of data.
        intercept (default = True) Specify wether or not to include an intercept term in the regressor.
        offset (default = False) Option to re-offset the regressed data at the estimated intercept
    Returns:
        eps (ndarray): Regressed data
    """
    if np.ndim(y)==1: #Make sure time series are presented as NX1 vectors
        y = y[:,None]
    N = np.shape(y)[0]
    if np.shape(x)[0] != N:
        raise Exception('Regressor must be of height N, with N the number of time points.')
    if np.ndim(x)==1: 
        x = x[:,None]
    if (offset and intercept) is False:
        intercept = True
    if intercept:
        x = np.insert(x,0,np.ones((1,N)),axis = 1)
    x_inv = np.linalg.pinv(x)
    beta = np.matmul(x_inv,y)
    y_est = np.matmul(x,beta)
    eps = y - y_est + np.mean(y)
    if offset:
        eps = eps+np.matmul(x[:,0][:,None],beta[0,:][None,:])
    return eps

In [19]:
def bin_pixels(frame, bin_size):
    height, width = frame.shape[:2]
    binned_height = height // bin_size
    binned_width = width // bin_size

    reshaped_frame = frame[:binned_height * bin_size, :binned_width * bin_size].reshape(binned_height, bin_size, binned_width, bin_size)
    binned_frame = np.sum(reshaped_frame, axis=(1, 3), dtype=np.float32)
    binned_frame = binned_frame / (bin_size**2)

    return binned_frame

In [33]:
def convert_to_hb(path_green, path_red, output_path):
    with tf.TiffFile(path_green) as tifG, tf.TiffFile(path_red) as tifR:
        R_green = tifG.pages
        R_red = tifR.pages
    
        num_frames = len(R_green) # Assuming the frames are along the first dimension
        frame_shape = R_green[0].asarray().shape
        stack_shape = (num_frames, frame_shape[0], frame_shape[1])

        lambda1 = 450 #nm
        lamba2 = 700 #nm
        npoints = 1000
        baseline_hbt = 100 #uM
        baseline_hbo = 60 #uM
        baseline_hbr = 40 #uM
        rescaling_factor = 1e6
        
        os.chdir(r"C:\Users\alexc\Documents\GitHub\Wide-Brain")
        eps_pathlength = ioi_epsilon_pathlength(lambda1, lamba2, npoints, baseline_hbt, baseline_hbo, baseline_hbr, filter=None)
        Ainv = np.linalg.pinv(eps_pathlength)*rescaling_factor
        os.chdir(current_path)

        with tf.TiffWriter(output_path+"\dHbO.tif", bigtiff=True) as writerHbO, tf.TiffWriter(output_path+"\dHbR.tif", bigtiff=True) as writerHbR, tf.TiffWriter(output_path+"\dHbT.tif", bigtiff=True) as writerHbT:
            for i in range(num_frames):
                data_G = R_green[i].asarray()
                data_R = R_red[i].asarray()
                ln_green = -np.log(data_G.flatten())
                ln_red = -np.log(data_R.flatten())
                ln_R = np.concatenate((ln_green.reshape(1, len(ln_green)), ln_red.reshape(1, len(ln_green))))
                Hbs = np.matmul(Ainv, ln_R)
                d_HbO = Hbs[0].reshape(frame_shape)
                d_HbR = Hbs[1].reshape(frame_shape)
                np.nan_to_num(d_HbO, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
                np.nan_to_num(d_HbR, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
                d_HbT = d_HbO + d_HbR
                
                writerHbO.write(np.float32(d_HbO), contiguous=True)
                writerHbR.write(np.float32(d_HbR), contiguous=True)
                writerHbT.write(np.float32(d_HbT), contiguous=True)

    return None

<>:23: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
C:\Users\gabri\AppData\Local\Temp\ipykernel_1404\1957387799.py:23: SyntaxWarning: invalid escape sequence '\d'
  with tf.TiffWriter(output_path+"\dHbO.tif", bigtiff=True) as writerHbO, tf.TiffWriter(output_path+"\dHbR.tif", bigtiff=True) as writerHbR, tf.TiffWriter(output_path+"\dHbT.tif", bigtiff=True) as writerHbT:
C:\Users\gabri\AppData\Local\Temp\ipykernel_1404\1957387799.py:23: SyntaxWarning: invalid escape sequence '\d'
  with tf.TiffWriter(output_path+"\dHbO.tif", bigtiff=True) as writerHbO, tf.TiffWriter(output_path+"\dHbR.tif", bigtiff=True) as writerHbR, tf.TiffWriter(output_path+"\dHbT.tif", bigtiff=True) as writerHbT:
C:\Users\gabri\AppData\Local\Temp\ipykernel_1404\1957387

In [21]:
def speckle_contrast(raw_speckle_data, window_size=7):
    num_frames, height, width = raw_speckle_data.shape
    contrast_data = np.zeros((num_frames, height, width))

    for frame_idx in range(num_frames):
        frame_data = raw_speckle_data[frame_idx]

        # Calculate local mean and standard deviation for the current frame
        local_mean = ndi.uniform_filter(frame_data, size=window_size)
        local_variance = ndi.uniform_filter(frame_data**2, size=window_size) - local_mean**2
        local_std = np.sqrt(local_variance)
        
        # Calculate speckle contrast for the current frame
        contrast_data[frame_idx] = local_std / local_mean

    return contrast_data

In [22]:
def bandpass_filter(data_3d, lowcut, highcut, fs=3.0, order=5):
    """
    Apply a band-pass filter to a 3D array along the 0th axis using vectorized operations and renormalize the amplitude.
    
    Parameters:
    - data_3d: the input 3D array (time, x, y)
    - lowcut: low cut-off frequency (default is 0.008 Hz)
    - highcut: high cut-off frequency (default is 0.09 Hz)
    - fs: sampling frequency (default is 3 Hz)
    - order: order of the filter (default is 5)
    
    Returns:
    - Filtered and renormalized 3D array.
    """
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    
    # Compute the max values of the original data for each pixel
    max_values = np.max(np.abs(data_3d), axis=0)
    
    # Reshape the 3D array into a 2D array
    data_2d = data_3d.reshape(data_3d.shape[0], -1)
    
    # Apply the filter to the entire 2D array along the 0th axis
    filtered_data_2d = filtfilt(b, a, data_2d, axis=0)
    
    # Reshape the 2D array back to its original 3D shape
    filtered_data_3d = filtered_data_2d.reshape(data_3d.shape)
    
    # Renormalize the amplitude for each pixel's time series
    max_values_filtered = np.max(np.abs(filtered_data_3d), axis=0)
    mask = max_values_filtered != 0  # Avoid division by zero
    filtered_data_3d[:, mask] *= (max_values[mask] / max_values_filtered[mask])
    
    return filtered_data_3d

## dHb pipeline

In [24]:
def compute_IOI(folder_path, save_path, red=True, green=True, baseline=[0, -1], bin_pixel=None, regression=False, filter_IOI=False):
    if red:
        # Create red channel and IOI_red.tif
        path_red = save_path + r"\IOI_red.tif"

        if len(identify_files(folder_path, ["rawdata_red"])) == 0:
            if len(identify_files(folder_path, ["redChan"])) == 0:
                create_channel(folder_path=folder_path, channel="red")
            data = np.load(folder_path+r"\redChan.npy").astype(np.float32)
            os.remove(folder_path+r"\redChan.npy")
            data /= 4096.0
            data = np.flip(data, axis=(1,2))
            tf.imwrite(folder_path + r"\rawdata_red.tif", data)
        elif len(identify_files(folder_path, ["rawdata_red"])) != 0:
            data = tf.imread(folder_path + r"\rawdata_red.tif")

        data = ndi.median_filter(data, size=[5, 1, 1]) 

        if regression:
            print("Global regression")
            regressor = np.mean(data, axis=(1,2))
            T, N, M = data.shape
            time_series = data.transpose(1, 2, 0).reshape(N * M, T)
            result = [regress_timeseries(series, regressor) for series in time_series]
            data = np.array(result).reshape(N, M, T).transpose(2, 0, 1)
            result, timeseries, regressor = None, None, None  # release memory

        norm_factor = np.mean(data[baseline[0]:baseline[1]], axis=0)
        if bin_pixel is not None:
            norm_factor = bin_pixels(norm_factor, bin_pixel)

        if filter_IOI:
            print("Filtering")
            data = bandpass_filter(data, lowcut=0.008, highcut=0.09, fs=3.0, order=3)

        print("Saving data")
        with tf.TiffWriter(path_red, bigtiff=True) as writer:
            for frame in data:
                if bin_pixel is not None:
                    frame = bin_pixels(frame, bin_pixel)
                frame = frame / norm_factor
                writer.write(np.float32(frame), contiguous=True)
        data = None
    
    if green:
        # Create green channel and IOI_green.tif
        path_green = save_path + r"\IOI_green.tif"

        if len(identify_files(folder_path, ["rawdata_green"])) == 0:
            if len(identify_files(folder_path, ["greenChan"])) == 0:
                create_channel(folder_path=folder_path, channel="green")
            data = np.load(folder_path+r"\greenChan.npy").astype(np.float32)
            os.remove(folder_path+r"\greenChan.npy")
            data /= 4096.0
            data = np.flip(data, axis=(1,2))
            tf.imwrite(folder_path + r"\rawdata_green.tif", data)
        elif len(identify_files(folder_path, ["rawdata_green"])) != 0:
            data = tf.imread(folder_path + r"\rawdata_green.tif")

        data = ndi.median_filter(data, size=[5, 1, 1]) 

        if regression:
            print("Global regression")
            regressor = np.mean(data, axis=(1,2))
            T, N, M = data.shape
            time_series = data.transpose(1, 2, 0).reshape(N * M, T)
            result = [regress_timeseries(series, regressor) for series in time_series]
            data = np.array(result).reshape(N, M, T).transpose(2, 0, 1)
            result, timeseries, regressor = None, None, None  # release memory

        norm_factor = np.mean(data[baseline[0]:baseline[1]], axis=0)
        if bin_pixel is not None:
            norm_factor = bin_pixels(norm_factor, bin_pixel)

        if filter_IOI:
            print("Filtering")
            data = bandpass_filter(data, lowcut=0.008, highcut=0.09, fs=3.0, order=3)

        print("Saving data")
        with tf.TiffWriter(path_green, bigtiff=True) as writer:
            for frame in data:
                if bin_pixel is not None:
                    frame = bin_pixels(frame, bin_pixel)
                frame = frame / norm_factor
                writer.write(np.float32(frame), contiguous=True)
        data = None
    
    return None

In [25]:
def dHb_pipeline(folder_path, save_path, baseline=[0, -1], bin_pixel=None, regression=False, filter_IOI=False, filter_dHb=False):
    # Create red channel and IOI_red.tif
    print("Load red channel")
    path_red = save_path + r"\IOI_red.tif"
    compute_IOI(folder_path, save_path, red=True, green=False, baseline=baseline, bin_pixel=bin_pixel, regression=regression, filter_IOI=filter_IOI)
          
    # Create green channel and IOI_green.tif
    print("Load green channel")
    path_green = save_path + r"\IOI_green.tif"
    compute_IOI(folder_path, save_path, red=False, green=True, baseline=baseline, bin_pixel=bin_pixel, regression=regression, filter_IOI=filter_IOI)
      
    # Create dHbO.tif, dHbR.tif and dHbT.tif
    print("Convert to dHb")
    convert_to_hb(path_green, path_red, save_path)
    
    if filter_dHb:
        print("Filtering dHb")
        Hb_paths = [r"\dHbO.tif", r"\dHbR.tif", r"\dHbT.tif"]
        for path in Hb_paths:
            data = tf.imread(folder_path + path)
            data = bandpass_filter(data, lowcut=0.008, highcut=0.09, fs=3.0, order=5)
            tf.imwrite(folder_path + path, np.float32(data))
    
    print("Done")

In [26]:
# CVR single file setup
setup = False
if setup:
    day = "S9"
    mouse = "RS_M35_S9"
    path = r"D:\CVR_dataset\{}\Dataset\{}".format(day, mouse)
    dHb_pipeline(folder_path=path, save_path=path, baseline=[0, -1], bin_pixel=2, regression=True, filter_IOI=False, filter_dHb=False)

In [27]:
# CVR multiple files setup
setup = False
if setup:
    all_paths = [r"D:\CVR_dataset\S7\Dataset", r"D:\CVR_dataset\S8\Dataset", r"D:\CVR_dataset\S9\Dataset", r"D:\CVR_dataset\S10\Dataset", r"D:\CVR_dataset\S11\Dataset"]
    for general_path in tqdm(all_paths):
        # CVR data
        keyword = ["CVR_M"]
        folders = identify_folders(general_path, keyword)
        for path in folders:
            print(path)
            try:
                dHb_pipeline(folder_path=path, save_path=path, baseline=[0, 90], bin_pixel=2, regression=False, filter_IOI=False, filter_dHb=False)
            except Exception as e:
                print(f"An error occurred with item {path}: {e}")
                continue
                
        # RS data
        keyword = ["RS_M"]
        folders = identify_folders(general_path, keyword)
        for path in folders:
            print(path)
            try:
                dHb_pipeline(folder_path=path, save_path=path, baseline=[0, -1], bin_pixel=2, regression=False, filter_IOI=False, filter_dHb=False)
            except Exception as e:
                print(f"An error occurred with item {path}: {e}")
                continue

In [31]:
# Single file HbT
setup = False
if setup:
    path = r"D:\GCaMP_testing2024\WF\06-06\Data\RS_M210"
    dHb_pipeline(folder_path=path, save_path=path, baseline=[0, -1], bin_pixel=2, regression=False, filter_IOI=False, filter_dHb=False)

In [ ]:
# Single file IOI
setup = False
if setup:
    path = r"D:\GCaMP_testing2024\WF\02-21\GCaMP_testing_v2"
    compute_IOI(folder_path=path, save_path=path, red=False, green=True, baseline=[0, -1], bin_pixel=None, regression=False, filter_IOI=False)

## LSCI pipeline

In [ ]:
def LSCI_pipeline(folder_path, save_path, background_path=None, baseline=None, regression=False, bin_pixel=True, filter_data=True):
    print("Load ir channel")
    if len(identify_files(folder_path, ["rawdata_ir"])) == 0:
        if len(identify_files(folder_path, ["irChan"])) == 0:
            create_channel(folder_path=folder_path, channel="ir")
        data = np.load(folder_path+r"\irChan.npy").astype(np.float32)
        os.remove(folder_path+r"\irChan.npy")
        data /= 4096.0
        data = np.flip(data, axis=(1,2))
        tf.imwrite(folder_path + r"\rawdata_ir.tif", data)
    elif len(identify_files(folder_path, ["rawdata_ir"])) != 0:
        data = tf.imread(folder_path + r"\rawdata_ir.tif")
    
    if background_path:
        print("Background substraction")
        if len(identify_files(background_path, ["irChan"])) == 0:
            create_channel(folder_path=background_path, channel=channel)
        data_bg = np.load(background_path+r"\irChan.npy").astype(np.float32)
        background = np.mean(data_bg / 4096.0, axis=0) 
        data -= background
        del data_bg, background
    
    if regression:
        print("Regress out")
        regressor = np.mean(data, axis=(1,2))
        T, N, M = data.shape
        time_series = data.transpose(1, 2, 0).reshape(N * M, T)
        del data
        result = [regress_timeseries(series, regressor) for series in time_series]
        data = np.array(result).reshape(N, M, T).transpose(2, 0, 1)
        del result, time_series
    
    print("Compute speckle")
    static_data = np.array([np.mean(data, axis=0)])
    static_data = speckle_contrast(static_data)
    if bin_pixel:
        static_data = bin_pixels(static_data[0], 2)
    
    data = speckle_contrast(data)
    data = 1 / np.power(data, 2)
    
    if baseline:
        norm_factor = np.mean(data[baseline[0]:baseline[1]], axis=0)
        data /= norm_factor
    
    if filter_data:
        data = ndi.gaussian_filter1d(data, sigma=2.0, axis=0)
    
    print("Saving to disk")
    with tf.TiffWriter(save_path + r"\LSCI.tif", bigtiff=True) as writer:
        for frame in data:
            if bin_pixel:
                frame = bin_pixels(frame, 2)
            writer.write(np.float32(frame), contiguous=True)

    tf.imwrite(save_path + r"\static_LSCI.tif", np.float32(static_data))
    del data, static_data

    return None

In [ ]:
# Single file setup
setup = False
if setup:
    day = "S5"
    mouse = "CVR_M34_{}".format(day)
    path = r"D:\CVR_dataset\{}\Dataset\{}".format(day, mouse)
    LSCI_pipeline(folder_path=path, save_path=path, background_path=None, baseline=None, bin_pixel=True, filter_data=False)

In [ ]:
setup = False
if setup:
    general_path = r"D:\CVR_dataset\S5\Dataset"
    keyword = ["CVR_M"]
    folders = identify_folders(general_path, keyword)
    for path in tqdm(folders):
        print(path)
        background_path = identify_folders(general_path, "bg_"+path[-7:-1])[0]
        LSCI_pipeline(folder_path=path, save_path=path, background_path=None, baseline=None, bin_pixel=True, filter_data=False)

In [ ]:
# LSCI testing
setup = False
if setup:
    general_path = r"D:\LSCI_testingv5"
    keyword = ["v"]
    folders = identify_folders(general_path, keyword)
    background_path = general_path + r"\background"
    for path in tqdm(folders):
        print(path)
        LSCI_pipeline(folder_path=path, save_path=path, background_path=None, baseline=None, bin_pixel=True, filter_data=False)

In [ ]:
# Single file LSCI
setup = False
if setup:
    path = r"D:\GCaMP_testing2024\WF\05-14\Data\RS_M210_P100"
    LSCI_pipeline(folder_path=path, save_path=path, background_path=None, baseline=None, bin_pixel=True, filter_data=False)

## Calcium imaging setup

In [ ]:
def baseline_minfilter(signal, window=300, sigma1=5, sigma2=100, debug=False):
    signal_flatstart = np.copy(signal)
    signal_flatstart[0] = signal[1]
    smooth = ndi.gaussian_filter1d(signal_flatstart, sigma1)
    mins = ndi.minimum_filter1d(smooth, window)
    baseline = ndi.gaussian_filter1d(mins, sigma2)
    if debug:
        debug_out = np.asarray([smooth, mins, baseline])
        return debug_out
    else:
        return baseline

    
def compute_dff_using_minfilter(timeseries, window=200, sigma1=0.1, sigma2=50):
    T, M, N = timeseries.shape
    dff = np.zeros((T, M, N))
    
    for m in range(M):
        for n in range(N):
            pixel_timeseries = timeseries[:, m, n]
            
            if np.any(pixel_timeseries):
                baseline = baseline_minfilter(pixel_timeseries, window=window, sigma1=sigma1, sigma2=sigma2)
                dff[:, m, n] = (pixel_timeseries - baseline) / baseline

    return dff


def hemodynamic_regression(folder_path, data_array):
    regressor_array = tf.imread(folder_path+r"\IOI_green.tif")
    regressor_array = regressor_array / np.mean(regressor_array, axis=0)
    
    regressed_data = data_array / regressor_array

    return regressed_data

In [ ]:
def calcium_pipeline(folder_path, save_path, baseline=[0, -1], regression=False, hemo_corr=False, bin_pixel=None, filter_data=True):
    print("Loading movie")
    channel = 'blue'
    path_blue = save_path + r"\Calcium_imaging.tif"
    
    if len(identify_files(folder_path, ["rawdata_blue"])) == 0:
        if len(identify_files(folder_path, ["blueChan"])) == 0:
            create_channel(folder_path=folder_path, channel="blue")
        data = np.load(folder_path+r"\blueChan.npy").astype(np.float32)
        os.remove(folder_path+r"\blueChan.npy")
        data /= 4096.0
        data = np.flip(data, axis=(1,2))
        tf.imwrite(folder_path + r"\rawdata_blue.tif", data)
    elif len(identify_files(folder_path, ["rawdata_blue"])) != 0:
        data = tf.imread(folder_path + r"\rawdata_blue.tif")
        
    # --- Test with transcranial data ---
#     data = np.float32(tf.imread(folder_path + r"\rawdata_gcamp.tif"))
    # ---
    
    data = ndi.median_filter(data, size=[5, 1, 1]) 
    
    if bin_pixel is not None:
        print("Pixel binning")
        binned_data = np.empty((data.shape[0], data.shape[1]//2, data.shape[2]//2), dtype=data.dtype)
        for i, frame in enumerate(data):
            binned_data[i] = bin_pixels(frame, bin_pixel)
        data = binned_data
        binned_data = None
    
    if hemo_corr:
        print("Hemodynamic regression")
        data /= np.mean(data, axis=0)
        data = hemodynamic_regression(folder_path, data)
    
    print("delta F/F calculation")
    data = compute_dff_using_minfilter(data, window=200, sigma1=0.9, sigma2=10)
        
    if regression:
        print("Global regression")
        regressor = np.mean(data, axis=(1,2))
        T, N, M = data.shape
        time_series = data.transpose(1, 2, 0).reshape(N * M, T)
        data = None  # release memory
        result = [regress_timeseries(series, regressor) for series in time_series]
        data = np.array(result).reshape(N, M, T).transpose(2, 0, 1)
        result, timeseries, regressor = None, None, None  # release memory
    
    if filter_data:
        data = ndi.gaussian_filter1d(data, sigma=0.9, axis=0)
        
    print("Saving data")
    with tf.TiffWriter(path_blue, bigtiff=True) as writer:
        for frame in data:
            writer.write(np.float32(frame), contiguous=True)
    data = None
    
    print("Done")

In [ ]:
# Test with transcranial data
setup = False
if setup:
    path = r"D:\Widefield_calcium_imaging\Data\rs-1-10min_2023-07-11\Geco3310\Geco3310_rs-1-10min_OI_2023-07-11"
    calcium_pipeline(path, path, baseline=None, bin_pixel=None, filter_data=True)

In [ ]:
setup = True
if setup:
    path = r"D:\GCaMP_testing2024\WF\06-06\Data\RS_M210"
    calcium_pipeline(path, path, regression=False, hemo_corr=True, bin_pixel=2, filter_data=False)

Loading movie
Pixel binning
Hemodynamic regression
delta F/F calculation
Saving data
Done


## Monitoring pipeline

In [ ]:
def convert_to_grayscale(input_file, output_file, frame_range):
    video = cv2.VideoCapture(input_file)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_range[1] == -1:
        frame_range[1] = total_frames
    output_file_template = output_file.replace('.tif', '_{}.tif')

    current_frame = frame_range[0]
    file_counter = 0
    max_frame = min(12000, frame_range[1]-frame_range[0])
    while current_frame < frame_range[1]:
        output_file = output_file_template.format(file_counter)
        print(output_file)
        with tf.TiffWriter(output_file, bigtiff=True) as tiff_writer:
            video.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
            for _ in tqdm(range(max_frame)):
                ret, frame = video.read()

#                 frame = frame[:, 420:1500]
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame = np.float32(frame)
                tiff_writer.write(frame, contiguous=True)
                
                current_frame += 1
                if current_frame > frame_range[1]:
                    break
                    
            file_counter += 1
            
    frame = None
    video.release()
    
    return None


def write_frame_difference(input_folder, input_file, output_file, frame_range, binning=False):
    video = cv2.VideoCapture(f"{input_folder}\\{input_file}")
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_range[1] == -1:
        frame_range[1] = total_frames - 1

    motion_energy = []
    frames_to_process = frame_range[1] - frame_range[0] + 1
    num_output_files = (frames_to_process + 11999) // 12000
    for file_counter in range(num_output_files):
        start_frame = frame_range[0] + file_counter * 12000
        end_frame = min(frame_range[0] + (file_counter + 1) * 12000 - 1, frame_range[1])
        num_frames_in_output = end_frame - start_frame + 1
        output_file_template = output_file.replace('.tif', f'_{file_counter}.tif')
        print(output_file_template)

        with tf.TiffWriter(output_file_template, bigtiff=True) as tiff_writer:
            video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            ret, frame1 = video.read()
            frame1 = cv2.cvtColor(frame1[:, 420:1500], cv2.COLOR_BGR2GRAY)
            for current_frame in tqdm(range(start_frame, end_frame)):
                ret, frame2 = video.read()
                frame2 = cv2.cvtColor(frame2[:, 420:1500], cv2.COLOR_BGR2GRAY)
                
                diff_frame = cv2.absdiff(frame2, frame1)
                if binning:
                    diff_frame = bin_pixels(diff_frame, 2)
                tiff_writer.write(np.uint8(diff_frame), contiguous=True)
                motion_energy.append(np.mean(diff_frame))
                
                frame1 = frame2

    video.release()
#     np.save(f"{input_folder}\\motion_energy", np.array(motion_energy))   
    
    return None


def compute_motion_energy(input_folder, input_file, frame_range):
    video = cv2.VideoCapture(f"{input_folder}\\{input_file}")
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_range[1] == -1:
        frame_range[1] = total_frames - 1

    motion_energy = []
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_range[0])
    ret, frame1 = video.read()
    frame1 = cv2.cvtColor(frame1[:, 420:1500], cv2.COLOR_BGR2GRAY)
    for current_frame in tqdm(range(frame_range[0], frame_range[1])):
        ret, frame2 = video.read()
        frame2 = cv2.cvtColor(frame2[:, 420:1500], cv2.COLOR_BGR2GRAY)

        diff_frame = cv2.absdiff(frame2, frame1)
        motion_energy.append(np.mean(diff_frame))

        frame1 = frame2

    video.release()
    np.save(f"{input_folder}\\motion_energy", np.array(motion_energy))   
    
    return None


def remove_artefacts(input_folder, threshold_lo):
    motion_energy_path = os.path.join(input_folder, 'motion_energy.npy')
    if os.path.exists(motion_energy_path):
        motion_energy = np.load(motion_energy_path)

        for idx, value in enumerate(motion_energy):
            if value < threshold_lo:
                if idx == 0:
                    motion_energy[idx] = (value + motion_energy[idx + 1]) / 2
                elif idx == len(motion_energy) - 1:
                    motion_energy[idx] = (motion_energy[idx - 1] + value) / 2
                else:
                    motion_energy[idx] = (motion_energy[idx - 1] + motion_energy[idx + 1]) / 2
        
        motion_energy = ndi.median_filter(motion_energy, size=5)
        np.save(motion_energy_path, motion_energy)
        motion_energy = None
    else:
        print(f"No 'motion_energy.npy' found in {input_folder}")

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.tif'):
            input_file = os.path.join(input_folder, file_name)

            with tf.TiffFile(input_file) as tif:
                frames = tif.asarray()
            
            for idx, frame in enumerate(frames):
                frame_mean = np.mean(frame)
                
                if frame_mean < threshold_lo:
                    if idx == 0:
                        frames[idx] = (frame + frames[idx + 1]) // 2
                    elif idx == len(frames) - 1:
                        frames[idx] = (frames[idx - 1] + frame) // 2
                    else:
                        frames[idx] = (frames[idx - 1] + frames[idx + 1]) // 2
            tf.imwrite(input_file, frames)
            frames = None
            
    return None

In [ ]:
setup = False
if setup:
    day = "S10"
    dataset = r"CVR_M33_{}".format(day)
    folder_path = r"D:\CVR_dataset\{}\Monitoring\{}_video".format(day, dataset)
    output_file = folder_path + r"\{}_monitoring.tif".format(dataset)
#     indices = np.load(folder_path+r"\indices.npy")
#     start = np.where(np.diff(indices) == 1.0)[0][0]
#     print(start)
#     frame_range = [start, start+int(12.5*60*30)]
    frame_range = [283, 283+int(12.5*60*30)]

#     write_frame_difference(folder_path, r"\{}_video.mp4".format(dataset), output_file, frame_range, binning=True)
    compute_motion_energy(folder_path, r"\{}_video.mp4".format(dataset), frame_range)
    remove_artefacts(folder_path, threshold_lo=0.5)

In [ ]:
# Multiple files setup
setup = False
if setup:
    general_path = r"D:\CVR_dataset\S9\Monitoring"
    keyword = ["RS_M"]
    length = int(10.0 * 60*30)
    folders = identify_folders(general_path, keyword)
    for path in folders:
        print(path)
        indices = np.load(path+r"\indices.npy")
        start = np.where(np.diff(indices) == 1.0)[0][0]
        frame_range = [start, start+length]
        
        dataset = identify_files(path, ".mp4")[0][:-10]
        output_file = path + r"\{}_monitoring.tif".format(dataset)
        
        # write_frame_difference(path, r"\{}_video.mp4".format(dataset), output_file, frame_range, binning=True)
        compute_motion_energy(path, r"\{}_video.mp4".format(dataset), frame_range)
        remove_artefacts(path, threshold_lo=0.5)

In [ ]:
setup = True
if setup:
    folder_path = r"D:\WF_with_cap\Monitoring\RS_M49_2_longexpred_video"
    input_file = folder_path + r"\RS_M49_2_longexpred_video.mp4"
    output_file = folder_path + r"\monitoring.tif"
    frame_range = [0, 400]

    convert_to_grayscale(input_file, output_file, frame_range)
#     write_frame_difference(folder_path, r"\{}_video.mp4".format(dataset), output_file, frame_range, binning=True)
#     compute_motion_energy(folder_path, r"\{}_video.mp4".format(dataset), frame_range)
#     remove_artefacts(folder_path, threshold_lo=0.5)

D:\WF_with_cap\Monitoring\RS_M49_2_longexpred_video\monitoring_0.tif


100%|██████████| 400/400 [00:44<00:00,  9.04it/s]


## Misc

In [ ]:
def time_binning(array):
    """
    Perform time binning on a 3D array of shape (T, M, N).

    Parameters:
    - array (numpy.ndarray): 3D array with shape (T, M, N).

    Returns:
    - numpy.ndarray: Binned 3D array with shape (T//5, M, N).
    """
    
    # Check if the input array has the right number of dimensions
    if len(array.shape) != 3:
        raise ValueError("Input array must be 3D")

    T, M, N = array.shape

    # If T is not divisible by 5, truncate it to the largest size that is divisible
    T_truncated = (T // 5) * 5
    array_truncated = array[:T_truncated]

    # Reshape the array to group every 5 frames along T axis
    reshaped_array = array_truncated.reshape(T_truncated//5, 5, M, N)

    # Sum along the second axis (which corresponds to the 5 frames to be binned)
    binned_array = reshaped_array.sum(axis=1)

    return binned_array

path = r"D:\Widefield_calcium_imaging\Data\dataset_test\dataset4"
# datasets = [r"\IOI_red.tif", r"\IOI_green.tif", r"\dHbO.tif", r"\dHbR.tif", r"\dHbT.tif"]
datasets = [r"\IOI_blue.tif"]
for dataset in tqdm(datasets):
    data = tf.imread(path+dataset)
    data = time_binning(data)
    tf.imwrite(path+dataset, data)

  0%|          | 0/1 [00:03<?, ?it/s]

KeyboardInterrupt



In [32]:
from scipy.signal import butter, filtfilt

def apply_butterworth_filter(array, lowcut, highcut, fs, order=4):
    """
    Apply a Butterworth filter to a 3D array along axis 0.
    
    Parameters:
        - array: 3D numpy array.
        - lowcut: Low cutoff frequency.
        - highcut: High cutoff frequency.
        - fs: Sampling frequency.
        - order: Order of the Butterworth filter (default is 4).
        
    Returns:
        - A 3D numpy array with the filter applied.
    """
    
    # Design the Butterworth filter
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    
    # Initialize an empty array to store the filtered data
    filtered_array = np.empty_like(array)
    
    # Apply the filter to each (M, N) slice along axis 0
    for i in range(array.shape[1]):
        for j in range(array.shape[2]):
            filtered_array[:, i, j] = filtfilt(b, a, array[:, i, j])
            
    return filtered_array

setup = False
if setup:
    path = r"D:\Widefield_calcium_imaging\Data\dataset_test\dataset1"
    dataset = r"\IOI_blue.tif"
    data = tf.imread(path+dataset)
    data = apply_butterworth_filter(data, lowcut=0.4, highcut=1.65, fs=3.36, order=4)
    tf.imwrite(path+dataset, data)

In [ ]:
# Apply mask
def apply_mask(data, mask):
    """
    Apply a 2D binary mask to a 3D array. All values in the 3D array outside of the mask are set to 0.0.
    
    Parameters:
        - data: 3D numpy array of shape (T, M, N).
        - mask: 2D binary array of shape (M, N). 
    
    Returns:
        - masked_data: 3D numpy array with values outside of the mask set to 0.0.
    """
    
    # Convert the binary mask to a boolean mask
    bool_mask = mask.astype(bool)
    
    # Initialize an array filled with zeros
    masked_data = np.zeros_like(data)
    
    # Apply the mask to the 3D data
    masked_data[:, bool_mask] = data[:, bool_mask]
    
    non_zero_coords = np.nonzero(masked_data)
    depth_range = (non_zero_coords[0].min(), non_zero_coords[0].max() + 1)
    row_range = (non_zero_coords[1].min(), non_zero_coords[1].max() + 1)
    col_range = (non_zero_coords[2].min(), non_zero_coords[2].max() + 1)
    masked_data = masked_data[depth_range[0]:depth_range[1], row_range[0]:row_range[1], col_range[0]:col_range[1]]
    
    return masked_data

path = r"D:\Widefield_calcium_imaging\Data\dataset_test\dataset1"
datasets = [r"\IOI_blue.tif", r"\IOI_red.tif", r"\IOI_green.tif", r"\dHbO.tif", r"\dHbR.tif", r"\dHbT.tif"]
mask_path = r"\mask.tif"
for dataset in tqdm(datasets):
    data = tf.imread(path+dataset)
    mask = tf.imread(path+mask_path)
    data = apply_mask(data, mask)
    tf.imwrite(path+dataset, data)